# PEtab benchmark model with BayesFlow

In [ ]:
# pip install git+https://github.com/Benchmarking-Initiative/Benchmark-Models-PEtab.git@master#subdirectory=src/python
# pypesto, amici, petab, fides, joblib

In [ ]:
import os

if "KERAS_BACKEND" not in os.environ:
    os.environ["KERAS_BACKEND"] = "jax"
else:
    print(f"Using '{os.environ['KERAS_BACKEND']}' backend")

import numpy as np
import pandas as pd
import pickle
from joblib import Parallel, delayed
from typing import Union
from collections import defaultdict
import matplotlib.pyplot as plt

import benchmark_models_petab as benchmark_models
import petab
import pypesto.optimize as optimize
import pypesto.sample as sample
import pypesto.petab
import pypesto.visualize as visualize
from pypesto.visualize.model_fit import visualize_optimized_model_fit
from scipy import stats

import bayesflow as bf

import amici
import logging
amici.swig_wrappers.logger.setLevel(logging.CRITICAL)
pypesto.logging.log(level=logging.ERROR, name="pypesto.petab", console=True)

from petab_helper import scale_values, values_to_linear_scale, amici_df_to_array, amici_pred_to_df, create_pypesto_problem

USE_MALA = False

# print all model names
print(benchmark_models.MODELS)

In [ ]:
# generate petab problem
#job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0))
n_cpus = 10 #int(os.environ.get('SLURM_CPUS_PER_TASK', 1))
problem_name = "Beer_MolBioSystems2014" #"Beer_MolBioSystems2014", "Boehm_JProteomeRes2014"
storage = '' # f'plots/{problem_name}/'
petab_problem = benchmark_models.get_problem(problem_name)

# decrease upper bounds for offset, scaling and noise parameters
scale_params_id = [name for name in petab_problem.parameter_df.index.values if name[:6] == 'offset' or name[:5] == 'scale']
petab_problem.parameter_df.loc[scale_params_id, 'upperBound'] = 100  # instead of 1000
sd_params_id = [name for name in petab_problem.parameter_df.index.values if name[:3] == 'sd_']
petab_problem.parameter_df.loc[sd_params_id, 'upperBound'] = 10  # instead of 1000


# add normal prior (on scale) around real parameters values
real_data_params = petab_problem.parameter_df.nominalValue
std = 0.5
for i in real_data_params.index:
    if petab_problem.parameter_df.loc[i, 'estimate'] == 0:
        continue
    # set prior mean depending on scale
    mean = scale_values(real_data_params.loc[i], petab_problem.parameter_df.loc[i, 'parameterScale'])
    if not 'objectivePriorType' in petab_problem.parameter_df or pd.isna(petab_problem.parameter_df.loc[i, 'objectivePriorType']):
        petab_problem.parameter_df.loc[i, 'objectivePriorType'] = "parameterScaleNormal"
        petab_problem.parameter_df.loc[i, 'objectivePriorParameters'] = f"{mean};{std}"

for i, row in petab_problem.parameter_df.iterrows():
    if 'objectivePriorType' in row and not pd.isna(row['objectivePriorType']):
        if row['estimate'] == 0:
            print(f"Parameter {i} has a {row['objectivePriorType']} prior but is not estimated, setting to nan")
            petab_problem.parameter_df.loc[i, 'objectivePriorType'] = np.nan
        # validate petab problem, if scale for parameter is defined, prior must be on the same scale
        if row['parameterScale'] != 'lin' and not row['objectivePriorType'].startswith('parameterScale'):
            raise ValueError(f"Parameter {i} has parameterScale {row['parameterScale']} but {row['objectivePriorType']} prior")


importer = pypesto.petab.PetabImporter(petab_problem, simulator_type="amici")
factory = importer.create_objective_creator()

model = factory.create_model(verbose=False)
amici_predictor = factory.create_predictor()
amici_predictor.amici_objective.amici_solver.setAbsoluteTolerance(1e-8)

# Creating the pypesto problem from PEtab
pypesto_problem = importer.create_problem()
pypesto_problem.print_parameter_summary()

In [ ]:
def get_samples_from_dict(samples_dict):
    samples = np.stack([samples_dict[name][..., 0] for name in pypesto_problem.x_names], axis=-1)
    return samples

def prior():
    lb = petab_problem.parameter_df['lowerBound'].values
    ub = petab_problem.parameter_df['upperBound'].values
    param_names_id = petab_problem.parameter_df.index.values
    param_scale = petab_problem.parameter_df['parameterScale'].values
    if 'objectivePriorType' in petab_problem.parameter_df.columns:
        prior_type = petab_problem.parameter_df['objectivePriorType'].values
    else:
        prior_type = [np.nan] * len(param_names_id)
    estimate_param = petab_problem.parameter_df['estimate'].values

    prior_dict = {}
    for i, name in enumerate(param_names_id):
        if estimate_param[i] == 0:
            prior_dict[name] = petab_problem.parameter_df['nominalValue'].values[i]  # linear space
        elif prior_type[i] == 'uniform':  # linear space
            prior_dict[name] = np.random.uniform(low=lb[i], high=ub[i])
        elif prior_type[i] == 'parameterScaleUniform' or pd.isna(prior_type[i]):
            # scale bounds to scaled space
            lb_scaled_i = scale_values(lb[i], param_scale[i])
            ub_scaled_i = scale_values(ub[i], param_scale[i])
            val = np.random.uniform(low=lb_scaled_i, high=ub_scaled_i)
            # scale to linear space
            prior_dict[name] = values_to_linear_scale(val, param_scale[i])
        elif prior_type[i] == 'parameterScaleNormal':
            mean, std = petab_problem.parameter_df['objectivePriorParameters'].values[i].split(';')
            lb_scaled_i = scale_values(lb[i], param_scale[i])
            ub_scaled_i = scale_values(ub[i], param_scale[i])
            a, b = (lb_scaled_i - float(mean)) / float(std), (ub_scaled_i - float(mean)) / float(std)
            rv = stats.truncnorm.rvs(loc=float(mean), scale=float(std), a=a, b=b)
            # scale to linear space
            prior_dict[name] = values_to_linear_scale(rv, param_scale[i])
        elif prior_type[i] == 'normal':
            mean, std = petab_problem.parameter_df['objectivePriorParameters'].values[i].split(';')
            a, b = (lb[i] - float(mean)) / float(std), (ub[i] - float(mean)) / float(std)
            rv = stats.truncnorm.rvs(loc=float(mean), scale=float(std), a=a, b=b)
            prior_dict[name] = rv
        elif prior_type[i] == 'laplace':
            loc, scale = petab_problem.parameter_df['objectivePriorParameters'].values[i].split(';')
            for t in range(10):
                rv = np.random.laplace(loc=float(loc), scale=float(scale))
                if lb[i] <= rv <= ub[i]:  # sample from truncated laplace
                    break
            prior_dict[name] = rv
        else:
            raise ValueError("Unknown prior type:", prior_type[i])
        # scale params and make list
        prior_dict[name] = np.array([scale_values(prior_dict[name], param_scale[i])])

    # prepare variables for simulation
    x = get_samples_from_dict(prior_dict)
    prior_dict['amici_params'] = x  # scaled parameters for amici
    return prior_dict

def simulator_amici(amici_params, return_df=False):
    pred = amici_predictor(amici_params)  # expect amici_params to be scaled
    sim_df, failed = amici_pred_to_df(pred, amici_params,
                                      factory=factory, petab_problem=petab_problem, pypesto_problem=pypesto_problem)
    sim = amici_df_to_array(sim_df)
    if failed:
        sim = sim * np.nan  # set all to nan if simulation failed
    if return_df:
        return dict(sim_data=sim, sim_failed=failed, sim_data_df=sim_df)
    return dict(sim_data=sim, sim_failed=failed)

In [ ]:
prior_sample = prior()
test = simulator_amici(prior_sample['amici_params'], return_df=True)
test['sim_data'].shape, prior_sample['amici_params'].shape, np.nansum(test['sim_data'])

In [ ]:
#test = simulator_amici(prior_sample['amici_params'], return_df=True)
#compute_objective(petab_problem, test['sim_data_df'], prior_sample['amici_params'])

In [ ]:
# # plot prior
# n_rows = len(pypesto_problem.x_names) // 6
# n_cols = int(np.ceil(len(pypesto_problem.x_names) / n_rows))
# fig, axs = plt.subplots(n_rows, n_cols, figsize=(2*n_rows, 2*n_cols), layout='constrained')
# axs = axs.flatten()
# samples = [prior() for i in range(1000)]
# for i, name in enumerate(pypesto_problem.x_names):
#     samples_i = np.array([s[name] for s in samples]).flatten()
#     axs[i].hist(samples_i, density=True)
#     axs[i].set_title(name)
#     # axs[i].axvline(scale_values(petab_problem.parameter_df['nominalValue'][i],
#     #                              petab_problem.parameter_df['parameterScale'][i]), color='red', linestyle='--')
#     # axs[i].axvline(scale_values(petab_problem.parameter_df['lowerBound'][i],
#     #                             petab_problem.parameter_df['parameterScale'][i]), color='blue', linestyle='--')
#     # axs[i].axvline(scale_values(petab_problem.parameter_df['upperBound'][i],
#     #                             petab_problem.parameter_df['parameterScale'][i]), color='blue', linestyle='--')
# plt.show()

In [ ]:
def run_mcmc(petab_problem, data_df=None, n_optimization_starts=0, n_chains=10, n_samples=10000,
             n_procs=10, use_mala=True, verbose=False, mix_start=0.75) -> Union[pypesto.result.Result, tuple[pypesto.result.Result, petab.Problem, pypesto.Problem]]:
    if data_df is None:
        # use true data
        _pypesto_problem = create_pypesto_problem(petab_problem)
        _petab_problem = None
    else:
        _measurement_df = data_df
        if not 'measurement' in _measurement_df.columns:
            _measurement_df['measurement'] = _measurement_df['simulation']  # pypesto expects measurement column
        _pypesto_problem, _petab_problem = create_pypesto_problem(petab_problem, _measurement_df)

    if n_optimization_starts == 0:
        print("Skipping optimization, sample start points for chains from prior")
        _result = None
        #x0 = [_pypesto_problem.get_reduced_vector(prior()['amici_params']) for _ in range(n_chains)]
    else:
        # do the optimization
        _result = optimize.minimize(
            problem=_pypesto_problem,
            optimizer=optimize.FidesOptimizer(verbose=0),
            #optimizer=optimize.ScipyOptimizer(method='L-BFGS-B'),
            n_starts=n_optimization_starts,
            engine=pypesto.engine.MultiProcessEngine(n_procs=n_procs) if n_procs > 1 else None,
            progress_bar=verbose
        )
        #x0 = _pypesto_problem.get_reduced_vector(_result.optimize_result.x[0])
        #x0 = [mix_start*np.array(x0[0]) + (1-mix_start)*_pypesto_problem.get_reduced_vector(prior()['amici_params'])]
        #if x0[0] is None:
        #    print("Warning: x0 contains nan, replace with prior sample")
        #    x0[0] = _pypesto_problem.get_reduced_vector(prior()['amici_params'])
        #x0 += [_pypesto_problem.get_reduced_vector(prior()['amici_params']) for _ in range(n_chains - 1)]

    _sampler = sample.AdaptiveParallelTemperingSampler(
        #internal_sampler=sample.MalaSampler() if use_mala else sample.AdaptiveMetropolisSampler(),
        internal_sampler=sample.AdaptiveMetropolisSampler(),
        n_chains=n_chains,
        options=dict(show_progress=verbose)
    )

    _result = sample.sample(
        problem=_pypesto_problem,
        n_samples=n_samples,
        sampler=_sampler,
        result=_result,
        #x0=x0,
    )
    sample.geweke_test(_result)

    if data_df is None:
        return _result
    return _result, _petab_problem, _pypesto_problem

In [ ]:
def get_mcmc_posterior_samples(res):
    burn_in = sample.geweke_test(res)
    if burn_in == res.sample_result.trace_x.shape[1]:
        print("Warning: All samples are considered burn-in.")
        _samples = res.sample_result.trace_x[0]  # only use first chain
    else:
        _samples = res.sample_result.trace_x[0, burn_in:]  # only use first chain
    #_samples = pypesto_problem.get_full_vector(_samples)
    #scales = petab_problem.parameter_df.loc[res.problem.x_names, 'parameterScale'].values
    #_samples = values_to_linear_scale(_samples, scales)
    return _samples

In [ ]:
n_optimization_starts = 1
test_params = prior()
test = simulator_amici(prior_sample['amici_params'], return_df=True)
#print(test_params)
new_result, new_petab_problem, new_pypesto_problem = run_mcmc(
    petab_problem=petab_problem,
    data_df=test['sim_data_df'],
    n_optimization_starts=n_optimization_starts,
    n_samples=1e2,
    n_procs=n_cpus,
    n_chains=2,
    verbose=True
)

if n_optimization_starts > 0:
    visualize.waterfall(new_result, size=(6, 4))
    visualize.parameters(new_result, size=(6, 25))
    sim_dict = visualize_optimized_model_fit(
        petab_problem=new_petab_problem,
        result=new_result,
        pypesto_problem=new_pypesto_problem,
        return_dict=True
    )
    print(test_params['amici_params']-new_result.optimize_result.x[0])

    fig, ax = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=False, figsize=(10, 3), layout='constrained')
    obs_name = ['Bac', 'Ind']
    for i, obs_id in enumerate(sim_dict['simulation_df']['observableId'].unique()):
        df_obs = sim_dict['simulation_df'][sim_dict['simulation_df']['observableId'] == obs_id]
        cmap = plt.get_cmap('tab20', len(df_obs['simulationConditionId'].unique()))
        for j, sim_con in enumerate(df_obs['simulationConditionId'].unique()):
            color = cmap(j)
            df = df_obs[df_obs['simulationConditionId'] == sim_con]
            ax[i].plot(df['time'].values, df['simulation'].values[:, 0], 'o', color=color,
                       markersize=0.7, label=f'Condition {j}')
        ax[i].set_ylabel(f'{obs_name[i]} [a.u.]', fontsize=14)
        ax[i].set_xlabel('Time [min]', fontsize=14)
        ax[i].tick_params(axis='x', labelsize=12)
        ax[i].spines['top'].set_visible(False)
        ax[i].spines['right'].set_visible(False)
    plt.savefig(f'plots/petab_benchmark_model_{problem_name}.png')
    plt.show()

visualize.sampling_fval_traces(new_result)
ax = visualize.sampling_parameter_traces(new_result)
ax = ax.flatten()
test_val = pypesto_problem.get_reduced_vector(test_params['amici_params'])
for i in range(new_pypesto_problem.dim):
    ax[i].axhline(test_val[i], color='red', linestyle='--')
print(np.mean((test_val-np.median(get_mcmc_posterior_samples(new_result), axis=0))**2))

In [ ]:
# n_optimization_starts = 20
# result = run_mcmc(
#     petab_problem=petab_problem,
#     pypesto_problem=pypesto_problem,
#     n_optimization_starts=n_optimization_starts,
#     n_samples=1e3
# )
#
# if n_optimization_starts > 0:
#     visualize.waterfall(result, size=(6, 4))
#     visualize.parameters(result, size=(6, 25))
#     visualize_optimized_model_fit(petab_problem=petab_problem, result=result, pypesto_problem=pypesto_problem);

In [ ]:
#ax = visualize.sampling_parameter_traces(result, size=(20, 20), full_trace=False, use_problem_bounds=False);
#visualize.sampling_scatter(result, size=(13, 6));

# Generate training and validation data

In [ ]:
from model_settings import MODELS, NUM_SAMPLES_INFERENCE, load_model

num_training_sets = 512 * 64
num_validation_sets = 1000

In [ ]:
@delayed
def sample_and_simulate(return_df=False):
    """Single iteration of sampling and simulation"""
    prior_sample = prior()
    test = simulator_amici(prior_sample['amici_params'], return_df=return_df)

    # Combine both dictionaries
    result = {**prior_sample, **test}
    return result


def simulate_parallel(n_samples, return_df=False):
    """Parallel sampling and simulation"""
    results = Parallel(n_jobs=n_cpus, verbose=100)(
        sample_and_simulate(return_df) for _ in range(n_samples)
    )
    results_dict = defaultdict(list)

    for r in results:
        for key, value in r.items():
            results_dict[key].append(value)
    for key, value_list in results_dict.items():
        if isinstance(value_list[0], pd.DataFrame):
            pass
        else:
           results_dict[key] = np.array(value_list)
    return results_dict

# MCMC workflow

In [ ]:
if os.path.exists(f"{storage}validation_data_petab_{problem_name}.pkl"):
    #with open(f'{storage}validation_data_petab_{problem_name}.pkl', 'rb') as f:
    #todo:    validation_data = pickle.load(f)
    with open(f'{storage}validation_data_petab_{problem_name}_100.pkl', 'rb') as f:
        validation_data = pickle.load(f)
    try:
        with open(f'{storage}training_data_petab_{problem_name}.pkl', 'rb') as f:
            training_data = pickle.load(f)
    except FileNotFoundError:
        training_data = None
        print("Training data not found")
else:
    training_data = simulate_parallel(num_training_sets)
    validation_data = simulate_parallel(num_validation_sets, return_df=True)

    with open(f'{storage}training_data_petab_{problem_name}.pkl', 'wb') as f:
        pickle.dump(training_data, f)
    with open(f'{storage}validation_data_petab_{problem_name}.pkl', 'wb') as f:
        pickle.dump(validation_data, f)

# # remove failed simulations
# if not training_data is None:
#     train_mask = ~training_data['sim_failed']
#     for key in training_data.keys():
#         training_data[key] = training_data[key][train_mask]
#     print(f"Failed Training data: {np.sum(~train_mask)} / {len(train_mask)}")
# val_mask = ~validation_data['sim_failed']
# for key in validation_data.keys():
#     if key == 'sim_data_df':
#         continue
#     validation_data[key] = validation_data[key][val_mask]
# print(f"Failed Validation data: {np.sum(~val_mask)} / {len(val_mask)}")

test_mean = np.nanmean(np.log(validation_data['sim_data'] + 1), axis=(0, 1), keepdims=True)
test_std = np.nanstd(np.log(validation_data['sim_data'] + 1), axis=(0, 1), keepdims=True)
print(validation_data['sim_data'].shape)

param_names = [name for i, name in enumerate(pypesto_problem.x_names) if i in pypesto_problem.x_free_indices]
lbs = np.array([lb for i, lb in enumerate(petab_problem.lb_scaled) if i in pypesto_problem.x_free_indices])
ubs = np.array([ub for i, ub in enumerate(petab_problem.ub_scaled) if i in pypesto_problem.x_free_indices])

In [ ]:
def run_mcmc_single(petab_prob, pypesto_prob, sim_data_df, n_starts,
                    n_mcmc_samples, n_final_samples, n_chains, use_mala):
    import amici
    import logging
    amici.swig_wrappers.logger.setLevel(logging.CRITICAL)
    pypesto.logging.log(level=logging.ERROR, name="pypesto.petab", console=True)

    if all(np.isnan(sim_data_df['simulation'])):
        return np.full((n_final_samples, len(pypesto_prob.x_free_indices)), np.nan)

    r, _, _ = run_mcmc(
        petab_problem=petab_prob,
        data_df=sim_data_df,
        n_optimization_starts=n_starts,
        n_samples=n_mcmc_samples,
        n_chains=n_chains,
        n_procs=1,
        use_mala=use_mala
    )

    if r is None:
        return np.full((n_final_samples, len(pypesto_prob.x_free_indices)), np.nan)

    ps = get_mcmc_posterior_samples(r)
    # num_samples random samples from posterior
    idx = np.random.choice(ps.shape[0], size=n_final_samples)
    return ps[idx]

In [ ]:
mcmc_path = f'{storage}mcmc_samples_{problem_name}{"_mala" if USE_MALA else ""}.pkl'
if os.path.exists(mcmc_path):
    with open(mcmc_path, 'rb') as f:
        mcmc_posterior_samples = pickle.load(f)
else:
    mcmc_posterior_samples = Parallel(n_jobs=n_cpus, verbose=10)(
        delayed(run_mcmc_single)(
            petab_prob=petab_problem,
            pypesto_prob=pypesto_problem,
            sim_data_df=sim_data_df,
            n_starts=10,
            n_mcmc_samples=1e5,
            n_final_samples=1000,
            n_chains=5,
            use_mala=USE_MALA
        ) for sim_data_df in validation_data['sim_data_df']
    )
    mcmc_posterior_samples = np.array(mcmc_posterior_samples)

    with open(mcmc_path, 'wb') as f:
        pickle.dump(mcmc_posterior_samples, f)
mcmc_mask = ~np.isnan(mcmc_posterior_samples.sum(axis=(1, 2)))
mcmc_posterior_samples_test = mcmc_posterior_samples

In [ ]:
fig = bf.diagnostics.recovery(
    estimates=mcmc_posterior_samples_test[mcmc_mask],
    targets=pypesto_problem.get_reduced_vector(validation_data['amici_params'].T).T[mcmc_mask],
    variable_names=param_names,
)
#fig.savefig(f"{storage}petab_benchmark_{problem_name}_mcmc_recovery.png")

fig = bf.diagnostics.calibration_ecdf(
    estimates=mcmc_posterior_samples_test[mcmc_mask],
    targets=pypesto_problem.get_reduced_vector(validation_data['amici_params'].T).T[mcmc_mask],
    variable_names=param_names,
    difference=True,
    stacked=True
)
#fig.savefig(f"{storage}petab_benchmark_{problem_name}_mcmc_calibration.png")

# BayesFlow workflow

In [ ]:
adapter = (
    bf.adapters.Adapter()
    .drop('amici_params')  # only used for simulation
    .drop('sim_data_df')
    .to_array()
    .convert_dtype("float64", "float32")
    .concatenate(param_names, into="inference_variables")
    .constrain("inference_variables", lower=lbs, upper=ubs, inclusive='both')  # after concatenate such that we can apply an array as constraint

    .as_time_series("sim_data")
    .log("sim_data", p1=True)
    #.standardize("sim_data", mean=test_mean, std=test_std)
    #.nan_to_num("sim_data", default_value=-3.0)
    .rename("sim_data", "summary_variables")
)

In [ ]:
# # check how the distributions look like
# test_params = adapter.forward(validation_data)['inference_variables']
#
# n_rows = len(param_names) // 6
# n_cols = int(np.ceil(len(param_names) / n_rows))
# fig, ax = plt.subplots(n_rows, n_cols, figsize=(2*n_rows, 2*n_cols), layout='constrained')
# ax = ax.flatten()
# for i, name in enumerate(param_names):
#     samples = test_params[:, i]
#     ax[i].hist(samples, density=True)
#     ax[i].set_title(name)
# plt.show()

In [ ]:
# check how the data distribution looks like (disable nan_to_num in adapter to see nans)
_test_data = adapter.forward(validation_data)['summary_variables']
# n_features = _test_data.shape[-1]
#
# n_rows = n_features // 5
# n_cols = int(np.ceil(n_features / n_rows))
# fig, ax = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(2*n_rows, 2*n_cols))
# ax = ax.flatten()
# for i in range(n_features):
#     ax[i].hist(_test_data[:, :, i].flatten(), density=True)
# plt.show()

In [ ]:
# print some stats about the data
print('Minimum', np.min(_test_data))
print('Maximum', np.max(_test_data))
print('Mean', np.mean(_test_data))
print('Standard Deviation', np.std(_test_data))
print('Nan Values', np.isnan(_test_data).sum())

In [ ]:
model_name = list(MODELS.keys())[4]
conf_tuple = MODELS[model_name]
print(model_name)

In [ ]:
%%time
workflow = load_model(adapter=adapter, conf_tuple=conf_tuple, param_names=param_names,
                      training_data=validation_data, #todo: training_data,
                      #use_ema=True,
                      validation_data=validation_data, storage=f'{storage}models/',
                      problem_name=problem_name, model_name=model_name)

In [ ]:
diagnostics_plots = workflow.plot_default_diagnostics(test_data=validation_data,
                                                      num_samples=10, #NUM_SAMPLES_INFERENCE,
                                                      approximator_kwargs=dict(rho=7, steps=30),
                                                      calibration_ecdf_kwargs={"difference": True, 'stacked': True})
#for k in diagnostics_plots.keys():
#    diagnostics_plots[k].savefig(f"{storage}petab_benchmark_{problem_name}_{model_name}_{k}.png")

In [ ]:
metrics_test = workflow.compute_default_diagnostics(test_data=validation_data,
                                     num_samples=100, #NUM_SAMPLES_INFERENCE,
                                     approximator_kwargs=dict(rho=7, steps=30)
                                    )
metrics_test.mean(axis=1)

In [ ]:
metrics_test = workflow.compute_default_diagnostics(test_data=validation_data,
                                     num_samples=100, #NUM_SAMPLES_INFERENCE,
                                     approximator_kwargs=dict(rho=3.5, steps=15)
                                    )
metrics_test.mean(axis=1)

# Compare MCMC and Diffusion models

In [ ]:
from bayesflow.diagnostics.metrics import root_mean_squared_error, posterior_contraction, calibration_error, classifier_two_sample_test
from visualize_results import plot_model_comparison_grid
from petab_helper import compute_likelihood_parallel

from scipy.stats import median_abs_deviation

In [ ]:
test_data = {}
for key, values in validation_data.items():
    if key == 'sim_data_df':
        test_data[key] = [v for i, v in enumerate(values) if mcmc_mask[i]]
    else:
        test_data[key] = values[mcmc_mask]
len(test_data['sim_data_df'])

In [ ]:
no_ema = False  # set to True, to get the noema model to validate training

In [ ]:
# add mcmc as baseline
if os.path.exists(f'{storage}metrics/petab_benchmark_{problem_name}_mcmc_{"_mala" if USE_MALA else ""}metrics.csv'):
    with open(f'{storage}metrics/petab_benchmark_{problem_name}_mcmc_{"_mala" if USE_MALA else ""}metrics.csv', 'rb') as f:
        mcmc_df = pd.read_csv(f, index_col=0)
else:
    print('Adding MCMC metrics')
    test_targets = get_samples_from_dict(test_data)

    rand_idx = np.random.choice(mcmc_posterior_samples_test.shape[1])
    workflow_samples_aug = compute_likelihood_parallel(petab_problem, mcmc_posterior_samples_test[mcmc_mask, rand_idx],
                                                       test_data, n_jobs=n_cpus)

    # augment test data
    test_data_aug = compute_likelihood_parallel(petab_problem, test_data['amici_params'], test_data,
                                                n_jobs=n_cpus)

    workflow_samples_aug = workflow_samples_aug[~np.isnan(workflow_samples_aug).any(axis=1)]
    test_data_aug = test_data_aug[~np.isnan(test_data_aug).any(axis=1)]
    print(f"{workflow_samples_aug.shape[0]} workflow samples and {test_data_aug.shape[0]} test data samples.")

    mcmc_df = pd.DataFrame([{
        'model': 'MCMC',
        'sampler': 'MCMC',
        'nrmse': root_mean_squared_error(
            mcmc_posterior_samples_test[mcmc_mask], test_targets, aggregation=np.nanmedian
        )['values'].mean(),
        'nrmse_mad': root_mean_squared_error(
            mcmc_posterior_samples_test[mcmc_mask], test_targets, aggregation=median_abs_deviation
        )['values'].mean(),
        'posterior_contraction': posterior_contraction(
            mcmc_posterior_samples_test[mcmc_mask], test_targets, aggregation=np.nanmedian
        )['values'].mean(),
        'posterior_contraction_mad': posterior_contraction(
            mcmc_posterior_samples_test[mcmc_mask], test_targets, aggregation=median_abs_deviation
        )['values'].mean(),
        'posterior_calibration_error': calibration_error(
            mcmc_posterior_samples_test[mcmc_mask], test_targets, aggregation=np.nanmedian
        )['values'].mean(),
        'posterior_calibration_error_mad': calibration_error(
            mcmc_posterior_samples_test[mcmc_mask], test_targets, aggregation=median_abs_deviation
        )['values'].mean(),
        'c2st': classifier_two_sample_test(workflow_samples_aug, test_data_aug, mlp_widths=(128, 128, 128),
                                           validation_split=0.25)
    }], index=[0])
    with open(f'{storage}metrics/petab_benchmark_{problem_name}_mcmc_{"_mala" if USE_MALA else ""}metrics.csv', 'wb') as f:
        mcmc_df.to_csv(f)

In [ ]:
for fusion_transformer_summary in [False, True]:
    metrics = []
    for i in range(len(MODELS)):
        if fusion_transformer_summary and not 'ft' in list(MODELS.keys())[i]:
            continue
        elif not fusion_transformer_summary and 'ft' in list(MODELS.keys())[i]:
            continue
        if os.path.exists(f'{storage}metrics/petab_benchmark_{problem_name}_metrics_{i}.pkl'):
            if no_ema and 'ema' in list(MODELS.keys())[i]:
                with open(f'{storage}metrics/petab_benchmark_{problem_name}_metrics_{i}_noema.pkl', 'rb') as f:
                    metric = pickle.load(f)
            else:
                with open(f'{storage}metrics/petab_benchmark_{problem_name}_metrics_{i}.pkl', 'rb') as f:
                    metric = pickle.load(f)
            metrics += metric
        else:
            print(f"Metrics for model {list(MODELS.keys())[i]} not found")

    # df, all columns to float beside model and sampler
    metrics_df = pd.DataFrame(metrics, index=None)
    metrics_df.index.name = None
    for col in ['nrmse', 'posterior_contraction', 'posterior_calibration_error', 'c2st']:
        metrics_df[col] = metrics_df[col].astype(float)
        metrics_df.loc[metrics_df[col].isna(), col] = 1
    metrics_df = pd.concat([metrics_df, mcmc_df], ignore_index=True)

    metrics_df.loc[metrics_df['nrmse'] == 1.0, 'posterior_contraction'] = np.nan  # samples were nan
    metrics_df.loc[metrics_df['nrmse'] == 1.0, 'posterior_contraction_mad'] = np.nan  # samples were nan
    metrics_df.loc[metrics_df['nrmse'] == 1.0, 'nrmse'] = np.nan  # samples were nan
    metrics_df.loc[metrics_df['nrmse'] == 1.0, 'nrmse_mad'] = np.nan  # samples were nan
    metrics_df = metrics_df[metrics_df['sampler'] != 'sde-pc']
    metrics_df['c2st'] = 0.5+np.abs(metrics_df['c2st']-0.5)
    metrics_df['rank'] = metrics_df['nrmse'] + metrics_df['posterior_calibration_error']

    if fusion_transformer_summary:
        metrics_df_2 = metrics_df.copy()
    else:
        metrics_df_1 = metrics_df.copy()
        metrics_df_joint = metrics_df.copy()

    display(metrics_df.sort_values(by='rank', inplace=False))

In [ ]:
# order from df1
order_map = (
    metrics_df_1[["model", "sampler"]]
    .drop_duplicates()
    .reset_index(drop=True)
    .assign(order=lambda x: range(len(x)))
    .rename(columns={"model": "model_key"})
)

# add model_key
df1 = metrics_df_1.copy()
df1["model_key"] = df1["model"]

df2 = metrics_df_2.copy()
df2["model_key"] = (
    df2["model"]
    .str.replace(r"_ft_ema$", "_ema", regex=True)
    .str.replace(r"_ft$", "", regex=True)
)

# combine
merged = pd.concat([df1, df2], ignore_index=True, sort=False)

# aggregate means ignoring NaN
agg = (
    merged.groupby(["model_key", "sampler"], dropna=False, as_index=False)
    .mean(numeric_only=True)
)

# choose display model name
display_names = df1[["model_key", "sampler", "model"]].drop_duplicates()
metrics_df_joint = agg.merge(display_names, on=["model_key", "sampler"], how="left")
metrics_df_joint["model"] = metrics_df_joint["model"].fillna(metrics_df_joint["model_key"])

# apply df1 order, unseen pairs go last
metrics_df_joint = (
    metrics_df_joint.merge(order_map, on=["model_key", "sampler"], how="left")
          .sort_values(["order"], na_position="last", kind="stable")
          .drop(columns=["order", "model_key"])
          .reset_index(drop=True)
)
# reorder columns so model and sampler come first
cols = ["model", "sampler"] + [c for c in metrics_df_joint.columns if c not in ["model", "sampler"]]
metrics_df_joint = metrics_df_joint[cols]
metrics_df_joint['rank'] = metrics_df_joint['nrmse'] + metrics_df_joint['posterior_calibration_error']
metrics_df_joint.sort_values(by='rank', inplace=False, ignore_index=True)

In [ ]:
np.corrcoef(metrics_df_joint[['posterior_calibration_error', 'c2st']].values.T)

In [ ]:
#plot_model_comparison_radar(metrics_df_joint, group_by_sampler=True, save_path=f'plots')
#plot_model_comparison_radar(metrics_df_joint, group_by_method='diffusion', save_path=f'plots')

In [ ]:
plot_model_comparison_grid(metrics_df_joint, save_path='plots')